In [1]:
import numpy as np
import pandas as pd

In [2]:
# Transaction Log 예시
pd.DataFrame(
    {
        "user_id": np.random.randint(1, 1000, 6),
        "item_id": np.random.randint(1, 1000, 6),
        "timestamp": pd.date_range(start="2025-01-01", periods=6),
        "action_type": np.random.choice(["view", "purchase"], 6),
        "duration": np.random.randint(100, 1000, 6),
    }
)

,user_id,item_id,timestamp,action_type,duration
0,505,125,2025-01-01,view,107
1,361,609,2025-01-02,purchase,579
2,129,424,2025-01-03,view,437
3,210,744,2025-01-04,purchase,833
4,98,423,2025-01-05,purchase,167
5,114,358,2025-01-06,view,200


In [3]:
# 직접적 피드백 예시: 영화 평점 데이터
num_users = 10
user_id = np.random.randint(1, 1000, num_users)
movie_id = np.random.randint(150, 200, num_users)

pd.DataFrame({"user_id": user_id, "movie_id": movie_id, "rating": np.random.choice([1, 2, 3, 4, 5], num_users)})

,user_id,movie_id,rating
0,265,184,1
1,420,157,4
2,852,160,2
3,63,188,4
4,834,161,4
5,978,169,3
6,538,153,3
7,63,185,1
8,953,193,5
9,900,193,3
